# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today into string to create the filename
format = "%m-%d-%Y"
today = datetime.now().strftime(format)
print(today)

03-01-2022


In [3]:
# Format yesterday into string to create the filename (in case today's data not ready yet)
yesterday = (datetime.now() - timedelta(days=1)).strftime(format)
print(yesterday)

02-28-2022


In [4]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-28-2022'

In [5]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-03-01 04:21:09,33.93911,67.709953,173659,7598,NaN,NaN,Afghanistan,446.099154,4.375241
1,NaN,NaN,NaN,Albania,2022-03-01 04:21:09,41.15330,20.168300,271563,3469,NaN,NaN,Albania,9436.479255,1.277420
2,NaN,NaN,NaN,Algeria,2022-03-01 04:21:09,28.03390,1.659600,264936,6835,NaN,NaN,Algeria,604.172631,2.579868
3,NaN,NaN,NaN,Andorra,2022-03-01 04:21:09,42.50630,1.521800,37999,151,NaN,NaN,Andorra,49180.094480,0.397379
4,NaN,NaN,NaN,Angola,2022-03-01 04:21:09,-11.20270,17.873900,98741,1900,NaN,NaN,Angola,300.432650,1.924226


Now immediately export the original jh dataset before making any changes

In [6]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [7]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,171931,172205,172441,172716,172901,173047,173084,173146,173395,173659
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,269904,270164,270370,270455,270734,270947,271141,271141,271527,271563
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,263936,264054,264201,264365,264488,264603,264706,264778,264855,264936


In [8]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [9]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7534,7539,7549,7569,7574,7578,7579,7585,7593,7598
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3433,3438,3445,3451,3453,3457,3458,3458,3464,3469
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6787,6797,6805,6812,6816,6820,6823,6828,6831,6835


In [10]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [11]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [13]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15420,15431,15436,15442,15451,15468,15479,15503,15509,15510
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,54734,54763,54784,54805,54837,54874,54904,54957,54972,54978
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5426,5429,5429,5430,5433,5436,5438,5440,5445,5445


In [14]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [15]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,184,184,184,184,184,192,194,194,194,194
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,635,635,635,635,636,640,640,640,640,640
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,92,92,92,92,92,92,93,93,93,93


In [16]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [17]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
674,1001.0,Autauga,Alabama,US,2022-03-01 04:21:09,32.539527,-86.644082,15510,194,NaN,NaN,"Autauga, Alabama, US",27761.370348,1.250806
711,1075.0,Lamar,Alabama,US,2022-03-01 04:21:09,33.779950,-88.096680,3644,63,NaN,NaN,"Lamar, Alabama, US",26396.233249,1.728869
712,1077.0,Lauderdale,Alabama,US,2022-03-01 04:21:09,34.901719,-87.656247,23910,366,NaN,NaN,"Lauderdale, Alabama, US",25784.813812,1.530740


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [18]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [19]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,79045043,950490,0.0
India,42931045,514023,0.0
Brazil,28796571,649676,0.0


In [20]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [21]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [22]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,79045043,950490,0.0,78094553.0,2022-03-01 04:21:09,37.934269,-91.444083
India,42931045,514023,0.0,42417022.0,2022-03-01 04:21:09,23.088275,81.806127
Brazil,28796571,649676,0.0,28146895.0,2022-03-01 04:21:09,-12.669522,-48.480493
France,22877926,139382,0.0,22738544.0,2022-03-01 04:21:09,6.430808,-34.730285
United Kingdom,19021076,161934,0.0,18859142.0,2022-03-01 04:21:09,30.654806,-30.248397


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [23]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [24]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,8978479,85468,0.0
Texas,6617106,84899,0.0
Florida,5844096,70084,0.0


In [25]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [26]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [27]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,8978479,85468,0.0,8893011.0,2022-03-01 04:21:09,37.843962,-120.728594
Texas,6617106,84899,0.0,6532207.0,2022-03-01 04:21:09,31.660643,-98.653069
Florida,5844096,70084,0.0,5774012.0,2022-03-01 04:21:09,28.940755,-82.700744
New York,4925221,67358,0.0,4857863.0,2022-03-01 04:21:09,42.544151,-75.474183
Illinois,3031220,35196,0.0,2996024.0,2022-03-01 04:21:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [28]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [29]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2796560,30716,0.0
Arizona,Maricopa,1246178,15761,0.0
Florida,Miami-Dade,1176706,10320,0.0
Illinois,Cook,1111190,14051,0.0
Texas,Harris,992400,10667,0.0


In [30]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [31]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [32]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2796560,30716,0.0,2765844.0,2022-03-01 04:21:09,34.308284,-118.228241,6037.0
Arizona,Maricopa,1246178,15761,0.0,1230417.0,2022-03-01 04:21:09,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1176706,10320,0.0,1166386.0,2022-03-01 04:21:09,25.611236,-80.551706,12086.0
Illinois,Cook,1111190,14051,0.0,1097139.0,2022-03-01 04:21:09,41.841448,-87.816588,17031.0
Texas,Harris,992400,10667,0.0,981733.0,2022-03-01 04:21:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [33]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [34]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [35]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,173146,271141,264778,37999,98701,11,7437,8893568,419423,3188117,...,838341,236174,19,514315,3219177,640214,514,11760,312611,235803
2022-02-27,173395,271527,264855,37999,98701,11,7437,8897178,419693,3209599,...,840511,236299,19,514315,3321005,640214,518,11769,312707,235803
2022-02-28,173659,271563,264936,37999,98741,11,7437,8900656,419832,3235017,...,842462,236405,19,515124,3443485,645947,521,11771,312750,236380


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [36]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [37]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,7585,3458,6828,151,1899,0,135,126048,8433,5157,...,6961,1630,1,5631,40050,5458,0,2133,3951,5393
2022-02-27,7593,3464,6831,151,1899,0,135,126120,8449,5171,...,6977,1633,1,5631,40144,5458,0,2134,3952,5393
2022-02-28,7598,3469,6835,151,1900,0,135,126152,8461,5210,...,6985,1635,1,5636,40522,5501,0,2135,3952,5395


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [38]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [39]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-02-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [40]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/19/22,2/20/22,2/21/22,2/22/22,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15420,15431,15436,15442,15451,15468,15479,15503,15509,15510
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3587,3588,3589,3592,3598,3613,3635,3644,3644,3644
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,23749,23761,23775,23793,23828,23861,23884,23899,23908,23910
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7491,7497,7500,7506,7515,7527,7536,7546,7561,7562
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,37434,37444,37447,37454,37472,37484,37497,37508,37512,37520


In [41]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [42]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [43]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,1279006,238683,84,1976890,818931,8961636,1311061,722377,256447,49,...,2004182,6612192,921956,112023,15395,1636510,1417498,488359,1570460,154909
2022-02-27,1279364,238683,84,1976890,819426,8961636,1311061,722377,256551,49,...,2004182,6613134,921956,112023,15395,1636510,1417498,488359,1570460,154909
2022-02-28,1279565,239494,84,1976890,819984,8978479,1313779,723389,256621,49,...,2004182,6617106,922852,112364,15398,1639942,1423990,490111,1571997,155101


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [44]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [45]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-02-26       15503   54957    5440  6374  14753    2278   5035   31636   
2022-02-27       15509   54972    5445  6375  14757    2279   5035   31637   
2022-02-28       15510   54978    5445  6375  14760    2279   5035   31638   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-26         8426     5073  ...    6696   1888     8014     1913   
2022-02-27         8430     5074  ...    6696   1888     8014     1913   
2022-02-28         8431     5074  ...    6702   1892     8023     1915   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-26          10991  9767  5616          0     2337   1541  
2022-02-27          10991  9767  5616          0     2337   1541  
2022-02-28          10999  9782  5623          0     2337   1548  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [46]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [47]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,18225,1163,0,27946,10465,85043,11752,10465,2709,0,...,24469,84775,4408,595,109,18536,11831,6308,13256,1718
2022-02-27,18225,1163,0,27946,10497,85043,11752,10465,2709,0,...,24469,84843,4408,595,109,18536,11831,6308,13256,1718
2022-02-28,18225,1163,0,27946,10524,85468,11772,10470,2709,0,...,24469,84899,4411,598,109,18771,11866,6317,13297,1736


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [48]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [49]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-02-26         194     640      93   99    218      49    119     588   
2022-02-27         194     640      93   99    218      49    119     588   
2022-02-28         194     640      93   99    218      49    119     588   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-02-26          152       78  ...     133     39       74       27   
2022-02-27          152       78  ...     133     39       74       27   
2022-02-28          152       78  ...     133     39       74       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-26            123    16    36          0       43     18  
2022-02-27            123    16    36          0       43     18  
2022-02-28            123    16    36         18       43     18  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [59]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [60]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [61]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,0.000358,0.000000,0.000272,0.0,0.000000,0.0,0.0,0.000630,0.001507,0.005533,...,0.003585,0.000585,0.0,0.001730,0.031688,0.000000,0.000000,0.000085,0.000759,0.001427
2022-02-27,0.001438,0.001424,0.000291,0.0,0.000000,0.0,0.0,0.000406,0.000644,0.006738,...,0.002588,0.000529,0.0,0.000000,0.031632,0.000000,0.007782,0.000765,0.000307,0.000000
2022-02-28,0.001523,0.000133,0.000306,0.0,0.000405,0.0,0.0,0.000391,0.000331,0.007919,...,0.002321,0.000449,0.0,0.001573,0.036880,0.008955,0.005792,0.000170,0.000138,0.002447


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [64]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,0.000792,0.000000,0.000733,0.0,0.000000,NaN,0.0,0.000715,0.002377,0.005067,...,0.001727,0.001229,0.0,0.000533,0.002202,0.000000,NaN,0.000939,0.001013,0.000185
2022-02-27,0.001055,0.001735,0.000439,0.0,0.000000,NaN,0.0,0.000571,0.001897,0.002715,...,0.002299,0.001840,0.0,0.000000,0.002347,0.000000,NaN,0.000469,0.000253,0.000000
2022-02-28,0.000659,0.001443,0.000586,0.0,0.000527,NaN,0.0,0.000254,0.001420,0.007542,...,0.001147,0.001225,0.0,0.000888,0.009416,0.007878,NaN,0.000469,0.000000,0.000371


In [65]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [66]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [67]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [70]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,0.000915,0.000000,0.0,0.000829,0.000990,0.000240,0.000000,0.000000,0.000429,0.0,...,0.00099,0.000705,0.000000,0.000000,0.002148,0.000000,0.00000,0.000000,0.000000,0.000000
2022-02-27,0.000280,0.000000,0.0,0.000000,0.000604,0.000000,0.000000,0.000000,0.000406,0.0,...,0.00000,0.000142,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2022-02-28,0.000157,0.003398,0.0,0.000000,0.000681,0.001879,0.002073,0.001401,0.000273,0.0,...,0.00000,0.000601,0.000972,0.003044,0.000195,0.002097,0.00458,0.003588,0.000979,0.001239


In [71]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [72]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [73]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-26      0.001550  0.000965  0.000368  0.000785  0.001290 -0.000877   
2022-02-27      0.000387  0.000273  0.000919  0.000157  0.000271  0.000439   
2022-02-28      0.000064  0.000109  0.000000  0.000000  0.000203  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-26      0.000397  0.002186  0.000000  0.000394  ...  0.000000   
2022-02-27      0.000000  0.000032  0.000475  0.000197  ...  0.000000   
2022-02-28      0.000000  0.000032  0.000119  0.000000  ...  0.000896   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-26      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2022-02-27      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2022-02-28      0.002119  0.001123  0.001045   0.000728  0.001536  0.001246   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2022-02-26            NaN      0.0  0.000000  
2022-02-27            NaN      0.0  0.000000  
2022-02-28            NaN      0.0  0.004543  

[3 rows x 3336 columns]

In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [75]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [76]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,0.002145,0.0,NaN,0.000537,0.003837,0.000741,0.000000,0.000000,0.001109,NaN,...,0.003898,0.001666,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2022-02-27,0.000000,0.0,NaN,0.000000,0.003058,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000802,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2022-02-28,0.000000,0.0,NaN,0.000000,0.002572,0.004997,0.001702,0.000478,0.000000,NaN,...,0.000000,0.000660,0.000681,0.005042,0.0,0.012678,0.002958,0.001427,0.003093,0.010477


In [77]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [78]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [79]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                   \
Admin2         Autauga Baldwin Barbour Bibb    Blount Bullock    Butler   
2022-02-26         0.0     0.0     0.0  0.0  0.004608     0.0  0.008475   
2022-02-27         0.0     0.0     0.0  0.0  0.000000     0.0  0.000000   
2022-02-28         0.0     0.0     0.0  0.0  0.000000     0.0  0.000000   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2022-02-26      0.001704 -0.006536      0.0  ...     0.0    0.0      0.0   
2022-02-27      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2022-02-28      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-02-26          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-02-27          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-02-28          0.0        0.0   0.0   0.0        inf      0.0    0.0  

[3 rows x 3336 columns]

In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [81]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [82]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,0.000495,0.000380,0.000357,0.000791,0.000031,0.0,0.000355,0.000910,0.001575,0.006387,...,0.004067,0.000617,0.046803,0.001360,0.028151,0.000889,0.002109,0.000267,0.000784,0.001693
2022-02-27,0.000966,0.000902,0.000324,0.000396,0.000016,0.0,0.000178,0.000658,0.001109,0.006563,...,0.003328,0.000573,0.023401,0.000680,0.029891,0.000444,0.004945,0.000516,0.000545,0.000847
2022-02-28,0.001244,0.000517,0.000315,0.000198,0.000210,0.0,0.000089,0.000524,0.000720,0.007241,...,0.002824,0.000511,0.011701,0.001127,0.033386,0.004700,0.005368,0.000343,0.000341,0.001647


In [83]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [84]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [85]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,0.000652,0.000356,0.000658,0.000211,1.407331e-07,NaN,3.833639e-06,0.000780,0.002331,0.006157,...,0.001952,0.001195,0.0,0.000587,0.002214,0.000865,NaN,0.000805,0.000540,0.000247
2022-02-27,0.000853,0.001046,0.000549,0.000105,7.036654e-08,NaN,1.916819e-06,0.000676,0.002114,0.004436,...,0.002125,0.001518,0.0,0.000293,0.002281,0.000433,NaN,0.000637,0.000397,0.000123
2022-02-28,0.000756,0.001245,0.000567,0.000053,2.633317e-04,NaN,9.584097e-07,0.000465,0.001767,0.005989,...,0.001636,0.001371,0.0,0.000591,0.005848,0.004155,NaN,0.000553,0.000198,0.000247


In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [87]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [88]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-02-26,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-27,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-02-28,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.49966,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [89]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [90]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [91]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,0.000916,0.001466,0.055812,0.000861,0.000825,0.000826,0.000541,0.000322,0.000797,1.580030e-215,...,0.001076,0.000845,0.000317,0.001087,0.001621,0.000601,0.000787,0.001026,0.000285,0.000436
2022-02-27,0.000598,0.000733,0.027906,0.000431,0.000715,0.000413,0.000271,0.000161,0.000601,7.900148e-216,...,0.000538,0.000494,0.000159,0.000543,0.000811,0.000301,0.000394,0.000513,0.000143,0.000218
2022-02-28,0.000378,0.002065,0.013953,0.000215,0.000698,0.001146,0.001172,0.000781,0.000437,3.950074e-216,...,0.000269,0.000547,0.000565,0.001794,0.000503,0.001199,0.002487,0.002050,0.000561,0.000729


In [92]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [93]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [94]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-02-26      0.001158  0.000768  0.000394  0.000605  0.001202 -0.000226   
2022-02-27      0.000773  0.000521  0.000656  0.000381  0.000737  0.000106   
2022-02-28      0.000419  0.000315  0.000328  0.000190  0.000470  0.000053   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-02-26      0.000404  0.001838  0.000156  0.000651  ...  0.000351   
2022-02-27      0.000202  0.000935  0.000315  0.000424  ...  0.000175   
2022-02-28      0.000101  0.000483  0.000217  0.000212  ...  0.000536   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-02-26      0.000355  0.000551 -0.000014   0.000381  0.000584  0.000303   
2022-02-27      0.000178  0.000276 -0.000007   0.000190  0.000292  0.000152   
2022-02-28      0.001148  0.000699  0.000519   0.000459  0.000914  0.000699   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-02-26           -1.0  0.000950  0.000491  
2022-02-27           -1.0  0.000475  0.000246  
2022-02-28           -1.0  0.000237  0.002394  

[3 rows x 3336 columns]

In [95]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [96]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [97]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-02-26,0.003169,0.001217,NaN,0.001429,0.003743,0.001728,0.000452,0.001006,0.000994,NaN,...,0.00536,0.002012,0.001159,0.001719,0.000302,0.002642,0.002905,0.002119,0.001573,0.000540
2022-02-27,0.001585,0.000608,NaN,0.000715,0.003401,0.000864,0.000226,0.000503,0.000497,NaN,...,0.00268,0.001407,0.000580,0.000859,0.000151,0.001321,0.001453,0.001060,0.000787,0.000270
2022-02-28,0.000792,0.000304,NaN,0.000357,0.002986,0.002931,0.000964,0.000490,0.000248,NaN,...,0.00134,0.001033,0.000630,0.002951,0.000076,0.007000,0.002205,0.001243,0.001940,0.005374


In [98]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [99]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [100]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2022-02-26      0.008095  0.000904  0.002748  0.000042  0.003487   
2022-02-27      0.004047  0.000452  0.001374  0.000021  0.001743   
2022-02-28      0.002024  0.000226  0.000687  0.000011  0.000872   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2022-02-26      3.137865e-09  0.009682  0.002921 -0.000755  0.003268  ...   
2022-02-27      1.568932e-09  0.004841  0.001461 -0.000377  0.001634  ...   
2022-02-28      7.844662e-10  0.002421  0.000730 -0.000189  0.000817  ...   

Province_State   Wyoming                                               \
Admin2              Park        Platte  Sheridan  Sublette Sweetwater   
2022-02-26      0.000479  5.019653e-08  0.000883  0.000009   0.000262   
2022-02-27      0.000239  2.509827e-08  0.000441  0.000005   0.000131   
2022-02-28      0.000120  1.254913e-08  0.000221  0.000002   0.000066   

Province_State                                                         
Admin2             Teton         Uinta Unassigned  Washakie    Weston  
2022-02-26      0.002083  5.493394e-08   0.752941  0.000744  0.001838  
2022-02-27      0.001042  2.746697e-08   0.752941  0.000372  0.000919  
2022-02-28      0.000521  1.373349e-08   0.752941  0.000186  0.000460  

[3 rows x 3336 columns]

In [101]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [102]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [103]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [104]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [105]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [106]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [107]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 